In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from st_dbscan import ST_DBSCAN
from coordinates import convert_to_utm

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

df = pd.read_csv("Clean_Data")


In [3]:
reduced_df = new[["Timestamp","Longitude", "Latitude"]]
reduced_df2 = reduced_df.copy()

In [4]:
convert_time = convert_to_utm(reduced_df, src_epsg=4326, dst_epsg=32633,
                        col_lat='Latitude', col_lon='Longitude')

Formal definition string for the old projection: proj=longlat datum=WGS84 no_defs ellps=WGS84 towgs84=0,0,0
Formal definition string for the new projection: proj=utm zone=33 datum=WGS84 units=m no_defs ellps=WGS84 towgs84=0,0,0


c:\Users\ysrmhmt\Desktop\Lectures\Data Mining\Project\Group_2\coordinates.py:16: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2, y2 = pyproj.transform(old_proj, new_proj, x1, y1)
c:\Users\ysrmhmt\Desktop\Lectures\Data Mining\Project\Group_2\coordinates.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[alias_lon] = x2
c:\Users\ysrmhmt\Desktop\Lectures\Data Mining\Project\Group_2\coordinates.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [5]:
convert_time.head()

,Timestamp,Longitude,Latitude
245820,2019-01-01 17:31:13.173884+03,3.002761e+06,3.481931e+06
797507,2019-01-01 03:11:06.838425+03,2.993272e+06,3.497359e+06
634177,2019-01-01 20:38:21.236842+03,3.002818e+06,3.468689e+06
12498,2019-01-01 12:16:46.318052+03,2.999867e+06,3.483780e+06
113047,2019-01-01 11:50:30.44247+03,3.001829e+06,3.482955e+06


In [6]:
# coordinates = [x for x in zip(df["Longitude"],df["Latitude"])]
# new = DBSCAN(eps=2/6371., min_samples=5, algorithm='ball_tree', metric='haversine').fit(np.radians(coordinates))


In [7]:
from sklearn.preprocessing import StandardScaler

we_dont_need_time_ = convert_time.drop(["Timestamp"], axis=1)
scaled_data = StandardScaler().fit_transform(we_dont_need_time_)

scaled_data = pd.DataFrame(scaled_data, columns=["Longitude", "Latitude"], index=convert_time.index)
a = reduced_df2["Timestamp"]
b = scaled_data

brand_new = pd.concat([a, b], axis=1)
# data = reduced_df[:, ]

brand_new['Timestamp'] = pd.to_datetime(brand_new['Timestamp'], format='%Y-%m-%d %H:%M:%S.%f%z')

# Convert the Pandas datetime objects to int64 representation
brand_new['Timestamp'] = brand_new['Timestamp'].astype('int64') / 10**9  # Convert nanoseconds to seconds

In [8]:
brand_new.head()

,Timestamp,Longitude,Latitude
245820,1.546353e+09,0.590513,-0.139080
797507,1.546301e+09,-1.678556,1.717220
634177,1.546364e+09,0.604005,-1.732377
12498,1.546334e+09,-0.101522,0.083448
113047,1.546333e+09,0.367434,-0.015888


In [9]:
st_dbscan3 = ST_DBSCAN(eps1 = 0.005, eps2 = 100, min_samples = 10)  # eps2 = 100 means almost 15 minutes.
st_dbscan3.fit(brand_new) 

In [10]:
value3 = pd.DataFrame(st_dbscan3.labels, index=reduced_df.index, columns=["Label2"])
sth_new = pd.concat([reduced_df2, value3], axis=1)

print(sth_new["Label2"].value_counts())

Label2
-1      351087
 9         176
 44        173
 72        162
 30        151
 34        131
 5         128
 50        127
 28        108
 55         96
 113        87
 46         82
 69         82
 2          75
 7          73
 17         73
 45         71
 35         68
 95         67
 77         65
 18         65
 12         64
 14         63
 93         62
 142        61
 88         56
 67         56
 190        52
 52         52
 37         51
 39         51
 57         49
 51         47
 258        47
 111        47
 223        46
 10         45
 246        45
 25         44
 116        42
 83         42
 171        42
 71         42
 278        42
 38         41
 31         41
 193        41
 96         41
 107        40
 68         39
 282        39
 61         39
 65         38
 130        38
 13         37
 19         36
 211        35
 265        35
 64         34
 267        34
 11         34
 225        33
 188        33
 74         33
 170        33
 73         32
 66

In [11]:
dropping = sth_new[sth_new["Label2"] == 9]

In [12]:
import plotly.express as px

px.set_mapbox_access_token("pk.eyJ1IjoieXNybWhtdGNsayIsImEiOiJjbHBuenJsbTcwbWJ1MmlwamxzMjRzOHo3In0.Mg3VHduKHGIe7B92ODMHzQ")
fig = px.scatter_mapbox(dropping, lat="Latitude", lon="Longitude",color="Label2",
                  color_continuous_scale=px.colors.sequential.Viridis, size_max=20,zoom=10)
fig.show()

In [13]:
dropping

,Timestamp,Longitude,Latitude,Label2
286156,2019-01-01 15:15:06.753073+03,28.956455,40.251895,9
182238,2019-01-01 15:11:54.371109+03,28.956340,40.251720,9
884318,2019-01-01 15:04:51.521953+03,28.956470,40.252090,9
190787,2019-01-01 15:11:47.344181+03,28.956170,40.251600,9
1187902,2019-01-01 15:17:10.187132+03,28.956540,40.251665,9
286163,2019-01-01 15:15:13.751099+03,28.956545,40.252000,9
150221,2019-01-01 15:05:18.104931+03,28.956450,40.251845,9
150267,2019-01-01 15:06:11.14665+03,28.956755,40.251855,9
884314,2019-01-01 15:04:11.196709+03,28.956765,40.252235,9
353715,2019-01-01 15:07:31.157998+03,28.956650,40.251970,9
